In [1]:
from pymfe.mfe import MFE

import csv
import json
import os
import math
import openpyxl
import re
import xlsxwriter
import zipfile
import rarfile
import itertools
import arfftocsv


import pandas as pd
import numpy as np

#external files
import file_operations
import pre_processing
import simple_characteristics
import quality_metrics
import information_theoretic

from simple_characteristics import class_index

In [2]:
listofFiles={}
abs_path = os.getcwd()+'/datasets/MFEAI/All feature'
file_operations.un_zip_files(abs_path) #extracted zipped files
arfftocsv.arffTocsv(abs_path)
test = 'test'
train = 'train'
valid = 'valid'
flag=0
duplicate_files = []
for path, subdirs, files in os.walk(abs_path):
    duplicate_files = []
    test_datasets = [s for s in files if test in s]
    train_datasets = [s for s in files if train in s]
    valid_datasets = [s for s in files if valid in s]
    if train_datasets:
        for td in train_datasets:
            pre_train = (td).split("_")[0]
            for ted in test_datasets:
                pre_test = (ted).split("_")[0]
                if(pre_train in ted) and (pre_test==pre_train):
                    table1 = pd.read_csv(path+"/"+td)
                    table2 = pd.read_csv(path+"/"+ted)
                    table1.columns = table2.columns
                    data = table1.append(table2)
                    data.to_csv(path+"/"+pre_train+".csv", index=False)
                    if flag:
                        files.append(pre_train+".csv")
                    else:
                        flag=1
                        files=[pre_train+".csv"]
    base = [os.path.splitext(f) for f in files]
    if(base):
        for a, b in itertools.combinations(base, 2):
            if(a[0]==b[0]):
                if((a[1]==".csv" and b[1]==".xlsx") or
                   (b[1]==".csv" and a[1]==".xlsx") or
                   (a[1]==".arff" and b[1]==".xlsx") or
                   (b[1]==".arff" and a[1]==".xlsx") or 
                   (a[1]==".arff" and b[1]==".csv") or
                   (b[1]==".arff" and a[1]==".csv") or 
                   (a[1]==".data" and b[1]==".csv") or
                   (b[1]==".data" and a[1]==".csv")):
                   duplicate_files.append(a[0]+".csv")

    if(duplicate_files):
        files = duplicate_files

    for name in files:
        if name.endswith((".data", ".csv", ".xlsx",".xls", ".asc",".dat",".trn")):
            listofFiles[name]=os.path.join(path, name)
            print(listofFiles[name])

/home/d19125691/Documents/Experiments/ontologyDCQ/onto-DCQ-FS_git/datasets/MFEAI/AdditionalError/Working/Chronic_Kidney_Disease/chronic_kidney_disease_full.csv
/home/d19125691/Documents/Experiments/ontologyDCQ/onto-DCQ-FS_git/datasets/MFEAI/AdditionalError/Working/Chronic_Kidney_Disease/chronic_kidney_disease.csv
/home/d19125691/Documents/Experiments/ontologyDCQ/onto-DCQ-FS_git/datasets/MFEAI/AdditionalError/Working/Autistic Spectrum Disorder Screening Data for Adolescent Data Set/Autism-Adolescent-Data.csv
/home/d19125691/Documents/Experiments/ontologyDCQ/onto-DCQ-FS_git/datasets/MFEAI/AdditionalError/Working/South German Credit (UPDATE) Data Set/SouthGermanCredit.asc
/home/d19125691/Documents/Experiments/ontologyDCQ/onto-DCQ-FS_git/datasets/MFEAI/AdditionalError/Working/Z-Alizadeh Sani Data Set/Z-Alizadeh sani dataset.xlsx
/home/d19125691/Documents/Experiments/ontologyDCQ/onto-DCQ-FS_git/datasets/MFEAI/AdditionalError/Working/User Knowledge Modeling Data Set/Data_User_Modeling_Datase

In [3]:
#Calculates all metrics
corrDict = {}
dataCharQuality = []

temp = []
count = []
categoryIdx = []

column_name_statistical = []
column_name_general = []
count=0
for eachFile in listofFiles:
    print("**************************************************************")
    print(eachFile)
    print(count) 
    count+=1
    print("----------------------")
    temp = []
    temp.append(eachFile) #file
    dataset = file_operations.custom_csv(listofFiles[eachFile])
    all_Label = simple_characteristics.get_labels(dataset)
    nUnique = simple_characteristics.count_unique_labels(dataset)
    missing_perc, missing_columns=quality_metrics.completeness(dataset)
    dataset = pre_processing.convert_str_int_categorical(dataset)
    dataset = pre_processing.convert_str_int_nominal(dataset)
    temp.append(nUnique)
    temp.append(missing_perc) #'completeness'
    temp.append(missing_columns) #miss_columns
    temp.append(quality_metrics.conciseness(dataset)) #conciseness
    acc, invalid_columns = quality_metrics.syntax_accuracy(dataset)
    temp.append(acc)  #syntax_accuracy
    temp.append(invalid_columns) #invalid_columns
   

    dataset = pre_processing.drop_rows(dataset) #drop rows with more than 75% NA
    dataset = pre_processing.drop_columns(dataset)  #drop columns with more than 75% NA
    dataset = pre_processing.convert_NAs(dataset)
    dataset = pre_processing.convert_str_int_categorical(dataset)
    dataset = pre_processing.convert_str_int_nominal(dataset)

    #statistical\n",
    X,y = simple_characteristics.get_XY(dataset)
    mfe = MFE(groups=["statistical"])
    mfe.fit(X.to_numpy())
    ft = mfe.extract()
    column_name_statistical = ft[0]
    for i in ft[1]:
        temp.append(i)

    #ML based metrics
    classes, ratio= quality_metrics.calculate_class_imbalance(dataset)
    temp.append(classes)  #ClassImbPoint
    temp.append(ratio)  #ClassImbRatio
    class_overlap_points, class_overlap_percentage, outlier_points, outliers=quality_metrics.compute_class_overlap(dataset)
    temp.append(class_overlap_points) #ClassOverlapPoints
    temp.append(class_overlap_percentage) #ClassOverlapPerc
    temp.append(outlier_points)  #OutlierPoints
    temp.append(outliers) #OutlierPerc
    labelIssues = quality_metrics.find_labelissues(dataset)
    temp.append(quality_metrics.find_labelissues(dataset)) #LabelIssues
    
    #simple
    mfe = MFE(groups=["general"])
    mfe.fit(X.to_numpy())
    ft = mfe.extract()
    column_name_general = ft[0]
    for i in ft[1]:
        temp.append(i)

    #info-theoretical
    mfe = MFE(groups=["info-theory"])

    mfe.fit(X.to_numpy())
    ft = mfe.extract()
    column_name_info = ft[0]
    for i in ft[1]:
        temp.append(i)
    #info-theoretical
    snrsd, snrmean, snrvalues = information_theoretic.signaltonoise(dataset)
    temp.append(snrmean)
    temp.append(snrsd)
    temp.append(snrvalues)
    temp.append(information_theoretic.ena_attributes(dataset))
    temp.append(information_theoretic.compute_class_entropy(dataset))
    
    #complexity
    mfe = MFE(groups=["complexity"])
    mfe.fit(X.to_numpy())
    ft = mfe.extract()
    column_name_complex = ft[0]
    for i in ft[1]:
        temp.append(i)

    dataCharQuality.append(temp)

**************************************************************
chronic_kidney_disease_full.csv
0
----------------------


/home/d19125691/.local/lib/python3.6/site-packages/pymfe/_internal.py:1566: UserWarning: It is not possible make equal discretization
  warnings.warn("It is not possible make equal discretization")


{0: 1.0, 2: 0.6048387096774194, 1: 0.008064516129032258} 3


/home/d19125691/.local/lib/python3.6/site-packages/sklearn/model_selection/_split.py:668: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/home/d19125691/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/d19125691/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (m

**************************************************************
chronic_kidney_disease.csv
1
----------------------


...

**************************************************************
Z-Alizadeh sani dataset.xlsx
4
----------------------


/home/d19125691/.local/lib/python3.6/site-packages/pymfe/_internal.py:1566: UserWarning: It is not possible make equal discretization
  warnings.warn("It is not possible make equal discretization")
/home/d19125691/.local/lib/python3.6/site-packages/pymfe/_internal.py:734: RuntimeWarning: Can't summarize feature 'cor' with summary 'sd'. Will set it as 'np.nan'.
  RuntimeWarning,
/home/d19125691/.local/lib/python3.6/site-packages/pymfe/_internal.py:734: RuntimeWarning: Can't summarize feature 'cor' with summary 'mean'. Will set it as 'np.nan'.
  RuntimeWarning,
/home/d19125691/.local/lib/python3.6/site-packages/scipy/stats/morestats.py:1678: UserWarning: Input data for shapiro has range zero. The results may not be accurate.
  warnings.warn("Input data for shapiro has range zero. The results "


{0: 1.0, 1: 0.4027777777777778} 2


/home/d19125691/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/home/d19125691/.local/lib/python3.6/site-packages/sklearn/linear_model/_logistic.py:765: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.h

**************************************************************
Data_User_Modeling_Dataset_Hamdi Tolga KAHRAMAN.csv
5
----------------------
{2: 1.0, 1: 0.9431818181818182, 0: 0.7159090909090909, 3: 0.2727272727272727} 4


In [ ]:
column_names = ['File','NumberOfUniqueClasses','Completeness','MissColumns','Conciseness','SyntaxAccuracy','InvalidColumns'] + list(column_name_statistical) + ['ClassImbPoint','ClassImbRatio', 'ClassOverlapPoints', 'ClassOverlapPerc', 'OutlierPoints', 'OutlierPerc','LabelIssues'] + list(column_name_general) + ['snr.mean','snr.sd','snr','ena','cEntropy'] + list(column_name_info) + list(column_name_complex)
df = pd.DataFrame(dataCharQuality, columns=column_names)
df.to_csv("Metafeatures.csv", index=False)